In [1]:
from bs4 import BeautifulSoup
import requests
import re
import csv

from datetime import datetime

import pandas as pd
import numpy as np

In [2]:
# Read values from obtained parsed apartment links in CSV format

with open('../01. URL_Data_Selection/apartments_links.csv', 'r') as f:
    reader = csv.reader(f, delimiter='\n')
    apartments_urls = [row[0] for row in reader]

In [3]:
russian_months = { 'янв': '01', 'фев': '02', 'мар': '03', 'апр': '04',
                   'май': '05', 'мая': '05', 'июн': '06', 'июл': '07',
                   'авг': '08', 'сен': '09', 'окт': '10', 'ноя': '11', 'дек': '12'
                 }

russian_labels_dict = { 'Цена': 'price', 'Адрес': 'address', 'Телефон': 'phone_number',
                        'Прочие контакты': 'other_contacts', 'Kоличество комнат': 'num_of_rooms',
                        'Общая площадь': 'total_area', 'Жилая площадь': 'living_area',
                        'Площадь кухни': 'kitchen_area', 'Этажность': 'floor_number',
                        'Тип дома/строение': 'house_type', 'Планировка': 'apt_layout',
                        'Расположение': 'arrangement', 'Комнаты': 'room_type',
                        'Санузел': 'bathroom_type', 'Балкон': 'balcony',
                        'Тип отопления': 'heating_type'                   
                      }

# Post-process month values according to russian_months dict

def process_month(month):
    if type(month) != str:
        return month
    else:        
        for key in russian_months.keys():
            if month.startswith(key):
                return russian_months[key]
            else:
                continue
                
# Handle null integer features

def integer_handler(integer):
    return int(integer) if integer else None 

# Handle null string features

def string_handler(string):
    return string if string else None 

In [4]:
%%time

# Calculate initial data fault / accuracy
total = len(apartments_urls)
errors = 0

# Final dataset object-feature matrix
dataset = []

for index, url in enumerate(apartments_urls):
    
    try:
    
        r = requests.get(url)

        soup = BeautifulSoup(r.content, 'html.parser')

        # Raw data selection section

        title = soup.find_all('h1')[-1].text
        podtitle = soup.find('div', {'class': 'podtitle'})
        desc = soup.find('div', {'class': 'topictext'}).text
        datepost = podtitle.find('span', {'class': 'datepost'}).text
        see = podtitle.find('span', {'class': 'see'}).text

        try:
            usersblockpod = soup.find('div', {'class': 'usersblockpod'})
            agency = usersblockpod.find('a').text

        except Exception:
            agency = 'Owner'

        if 'сегодня' in datepost:
            post_day = str(datetime.today().day)
            post_month = str(datetime.today().month)

        else:    
            post_day = str(re.findall(r'\d+', datepost)[0])
            post_month = process_month(re.sub('[0-9]+','', datepost).strip())

        table = soup.find('table', {'class': 'dots'})
        data_rows = table.find_all('tr')

        ths = []
        tds = []
       
        for tr in data_rows:
            
            th_tag = tr.find('th')
            td_tag = tr.find('td')
            ths.append(th_tag)
            tds.append(td_tag)            

            building_href = td_tag.find('a', attrs={'href': True})
            if building_href != None:
                building_link = 'https:' + str(building_href['href'])
  
      
        index_del = ths.index(None)
        tds.pop(index_del)
        ths.pop(index_del)

        ths = [col.text.strip() for col in ths if col]
        tds = [col.text.strip() for col in tds if col]

        apartment_data = dict(zip(ths, tds))


        try:        
            different_ads = soup.find('table', {'class': 'map'})
            ads_rows = different_ads.find_all('tr')
            num_of_ads_in_this_house = len(ads_rows) - 1

        except Exception:
            num_of_ads_in_this_house = 0

        try:        
            images_section = soup.find_all('a', {'data-fancybox': 'adv-mainws'})
            num_of_images = len(images_section)

        except Exception:
            num_of_images = 0

        # Features section

        views = integer_handler(see)    
        post_day = string_handler(post_day)
        post_month = string_handler(post_month)
        desc = string_handler(desc)
        title = string_handler(title)
        agency = string_handler(agency)
        num_of_ads_in_this_house = integer_handler(num_of_ads_in_this_house)
        num_of_images = integer_handler(num_of_images)
        building_link = string_handler(building_link)

        apartment = { 
                        'day_posted': post_day,
                        'month_posted': post_month,
                        'title': title,
                        'agency': agency,
                        'num_of_views': views,
                        'desc': desc,
                        'num_of_ads_in_this_house': num_of_ads_in_this_house,
                        'num_of_images': num_of_images,
                        'building_link': building_link
                    }
        
        apartment = {**apartment, **apartment_data}

        dataset.append(apartment)

        if index > 0 and index % 500 == 0:
            print(f'{index} dataset objects processed')
            
    except Exception as e:
        errors += 1
        continue

500 dataset objects processed
1000 dataset objects processed
1500 dataset objects processed
2000 dataset objects processed
2500 dataset objects processed
3000 dataset objects processed
3500 dataset objects processed
4000 dataset objects processed
4500 dataset objects processed
5000 dataset objects processed
5500 dataset objects processed
6000 dataset objects processed
6500 dataset objects processed
7000 dataset objects processed
7500 dataset objects processed
CPU times: user 15min 37s, sys: 9.59 s, total: 15min 46s
Wall time: 52min 44s


In [21]:
print(f'Number of Faulty Objects is: {errors} out of {len(dataset)}')
print(f'Avg rate of processing is: {3164 / len(apartments_urls)} URL per second')


print(f'Dataset creation Accuracy Rate: {(len(dataset) - errors) * 100 /  len(dataset):.3f}%')
print(f'Dataset creation Error Rate: {errors * 100 /  len(dataset):.3f}%')

Number of Faulty Objects is: 142 out of 7651
Avg rate of processing is: 0.4060053894520724 URL per second
Dataset creation Accuracy Rate: 98.144%
Dataset creation Error Rate: 1.856%


In [22]:
housing_data = pd.DataFrame(dataset)
housing_data.head()

,day_posted,month_posted,title,agency,num_of_views,desc,num_of_ads_in_this_house,num_of_images,building_link,Цена,...,Площадь кухни,Этажность,Тип дома/строение,Планировка,Расположение,Комнаты,Санузел,Балкон,Тип отопления,Адрес недвижимости
0,13,10,Недвижимость / Продам - четырехкомнатную кварт...,"Оператор недвижимости ""Перспектива24""",136,Срочно продается четырёхкомнатная квартира по ...,9.0,1.0,https://cheb.ru/doma/flukina/18.xhtml,6 588 275 руб. (60 112 руб/кв.м.),...,17.8 кв.м,1/9,Кирпичный,Новая,Угловая,Раздельные,Раздельный,Балкон,Индивидуальное отопление,NaN
1,15,10,Недвижимость / Продам - трехкомнатную квартиру...,"Центр И.В. Шумова ""Фамилия""",272,Продается трехкомнатная квартира в отличном со...,2.0,26.0,https://cheb.ru/doma/talvira/4.xhtml,3 550 000 руб. (50 426 руб/кв.м.),...,NaN,4/9,Монолитный,NaN,NaN,NaN,NaN,NaN,Центральное,NaN
2,6,10,Недвижимость / Продам - трехкомнатную квартиру...,Owner,133,"продаю квартиру в Канаше, требует ремонта",5.0,NaN,https://cheb.ru/chuvashia/kanash/kanash.htm,900 000 руб. (16 071 руб/кв.м.),...,NaN,3/5,Кирпичный,Новая,Односторонняя,Раздельные,Раздельный,Лоджия,Центральное,NaN
3,11,10,Недвижимость / Продам - однокомнатную квартиру...,"Агентство недвижимости ""ЧАР""",3267,Продаю 1- комнатную квартиру в новом доме. Отл...,20.0,8.0,https://cheb.ru/doma/rozhdestvenskogo/8.xhtml,2 150 000 руб. (56 579 руб/кв.м.),...,10 кв.м,10/16,Панельный,Новая,Односторонняя,NaN,Совмещенный,Лоджия,NaN,NaN
4,13,10,Недвижимость / Продам - однокомнатную квартиру...,"Агентство недвижимости ""Ориентир""",54,Отличное предложение!!! Теплая уютная квартира...,2.0,3.0,https://cheb.ru/doma/maksimova/8_10.xhtml,1 400 000 руб. (44 444 руб/кв.м.),...,5.6 кв.м,3/5,Кирпичный,Хрущевка,Односторонняя,Раздельные,Совмещенный,Балкон,Центральное,NaN


In [23]:
housing_data.to_csv('housing_data.csv')